In [1]:
import pandas as pd
from matplotlib.projections import projection_registry

import project_fitness_function as fit

In [2]:
file = 'train_data.csv'
terreno_hectares = 2
limite_agua = 8.5
janela_dias = 90
orcamento_maximo = 7.5

interacoes = 250

In [3]:
df = pd.read_csv(file)

In [4]:
m2_area = terreno_hectares * 10_000

In [5]:
best_result = []

def xpto(samples):
    results = []
    for i, sample in enumerate(samples):
        fit_result = fit.project_fitness_function(sample, orcamento_maximo, m2_area, limite_agua, janela_dias)
        results.append({'sample': sample, 'fitness': fit_result})

    results.sort(key=lambda x: x['fitness'],reverse=True)

    if len(best_result) == 0:
        best_result.append(results[0])

    if results[0]['fitness'] > best_result[0]['fitness']:
        best_result[0] = results[0]




# população inicial
initial_population = []

for i in range(100):
    initial_population.append(df.sample(3))

# executa para população inicial
xpto(initial_population)

#crossover de genes
for i in range(interacoes):
    new_population = fit.crossover(initial_population,df)
    xpto(new_population)

print(best_result)

[{'sample':     CULTURA  CUSTO PRODUÇÃO  PRODUTIVIDADE  CICLO DE VIDA MIN EM DIAS  \
9     trigo             2.0            2.0                        100   
2  mandioca             1.5            2.5                         91   
8    feijão             1.5            2.0                         60   

   CICLO DE VIDA MAX EM DIAS  REQUISITO DE ÁGUA  RETORNO FINANCEIRO SINERGIA  \
9                        130                2.0                 2.0     soja   
2                         18                1.5                 2.0   feijão   
8                         90                2.0                 2.0    milho   

       COMPANHEIRA        NEUTRA               ANTAGÔNICA  ESPAÇO MÍNIMO m²  
9      arroz,aveia  milho,feijão  mandioca,cana-de-açúcar              0.15  
2     milho,batata    sorgo,café            trigo,algodão              1.00  
8  mandioca,batata    soja,trigo   algodão,cana-de-açúcar              0.40  , 'fitness': 63.497405672279264}]
